In [2]:
import os
import pandas as pd
import numpy as np
import pyodbc
from datetime import datetime, timedelta
import json
#from Functions import return_jalali_date, user_pyodbc_connection, pyodbc_connection
#from Functions import Node1va2_Username, Node1va2_Password
from Functions import *
import jdatetime
from datetime import timedelta
import jdatetime
import requests
from datetime import timedelta, datetime
from persiantools.jdatetime import JalaliDate
import re
from dotenv import load_dotenv

In [4]:
from openai import OpenAI

client = OpenAI(
    api_key="local-api-key",   # هرچیزی، مهم نیست
    base_url="http://127.0.0.1:1234/v1"
)

In [5]:
def classify_item(text: str) -> str:
    response = client.chat.completions.create(
        
        model="qwen3-4b-instruct-2507",  # مثلا: mistral, llama3, qwen
        temperature=0,
        messages=[
            {"role": "user", "content": text}]
    )
    return response.choices[0].message.content.strip()


In [11]:
ScrapingData = pd.read_excel('Out2.xlsx')

In [12]:
ScrapingData

,biography,followers,following,full_name,id,is_private,is_verified,posts,profile_pic_url,username
0,فروشگاه آنلاین طلا و نقره,23,1222,فروشگاه آنلاین طلا و نقره,chertopert1,False,False,450,absd1,abs1
1,عطر و لوازم آرایشی,3334,1,عطر و لوازم آرایشی,chertopert2,False,False,32,absd2,abs2
2,عطر و لوازم آرایشی,34,1,عطر و لوازم آرایشی,chertopert3,False,False,432,absd3,abs3
3,فروشگاه لباس زنانه,5677,12,فروشگاه لباس زنانه,chertopert4,False,False,57,absd4,abs4


In [15]:
ScrapingGroups = []

for i in  ScrapingData.biography:
    ScrapingGroups.append(classify_item(i))

In [17]:
pd.DataFrame(ScrapingGroups , columns = ['Master'])

,Master
0,طلا
1,زیبایی و سلامت
2,زیبایی و سلامت
3,مد و پوشاک


In [21]:
ScrapingData['Master'] = pd.DataFrame(ScrapingGroups)

In [22]:
ScrapingData

,biography,followers,following,full_name,id,is_private,is_verified,posts,profile_pic_url,username,Master
0,فروشگاه آنلاین طلا و نقره,23,1222,فروشگاه آنلاین طلا و نقره,chertopert1,False,False,450,absd1,abs1,طلا
1,عطر و لوازم آرایشی,3334,1,عطر و لوازم آرایشی,chertopert2,False,False,32,absd2,abs2,زیبایی و سلامت
2,عطر و لوازم آرایشی,34,1,عطر و لوازم آرایشی,chertopert3,False,False,432,absd3,abs3,زیبایی و سلامت
3,فروشگاه لباس زنانه,5677,12,فروشگاه لباس زنانه,chertopert4,False,False,57,absd4,abs4,مد و پوشاک


In [14]:
import pandas as pd
import numpy as np
import datetime as dt

# تعداد مشتری و تراکنش
num_customers = 20
num_transactions = 50

# ساخت داده ساختگی
np.random.seed(42)  # برای تکرارپذیری

customer_names = [f'Customer_{i+1}' for i in range(num_customers)]
transaction_ids = [f'TX_{i+1}' for i in range(num_transactions)]

df = pd.DataFrame({
    'Transaction_ID': np.random.choice(transaction_ids, size=num_transactions),
    'Date': [dt.datetime(2025, 1, 1) + dt.timedelta(days=int(x)) for x in np.random.randint(0, 365, size=num_transactions)],
    'CustomerID': np.random.choice(customer_names, size=num_transactions),
    'Product': np.random.choice(['Product_A', 'Product_B', 'Product_C'], size=num_transactions),
    'Total_Items': np.random.randint(1, 10, size=num_transactions),
    'TotalPrice': np.random.randint(10, 500, size=num_transactions),
    'Payment_Method': np.random.choice(['Cash', 'Card'], size=num_transactions),
    'City': np.random.choice(['City1', 'City2', 'City3'], size=num_transactions),
    'Store_Type': np.random.choice(['Online', 'Offline'], size=num_transactions),
    'Discount_Applied': np.random.choice([0, 5, 10], size=num_transactions),
    'Customer_Category': np.random.choice(['Regular', 'VIP'], size=num_transactions),
    'Season': np.random.choice(['Winter', 'Spring', 'Summer', 'Fall'], size=num_transactions),
    'Promotion': np.random.choice(['Promo1', 'Promo2', 'None'], size=num_transactions)
})

print(df.head())


  Transaction_ID       Date  CustomerID    Product  Total_Items  TotalPrice  \
0          TX_39 2025-02-22  Customer_8  Product_B            2          69   
1          TX_29 2025-12-06  Customer_3  Product_A            2         378   
2          TX_15 2025-04-02  Customer_3  Product_B            6          11   
3          TX_43 2025-09-21  Customer_1  Product_A            7         394   
4           TX_8 2025-02-04  Customer_5  Product_A            5         313   

  Payment_Method   City Store_Type  Discount_Applied Customer_Category  \
0           Card  City2    Offline                 0               VIP   
1           Cash  City1     Online                10           Regular   
2           Card  City2    Offline                 5           Regular   
3           Cash  City1     Online                 0               VIP   
4           Card  City2    Offline                 5               VIP   

   Season Promotion  
0    Fall    Promo2  
1    Fall    Promo1  
2    Fall     

In [15]:
# تاریخ مرجع برای Recency
NOW = df['Date'].max() + dt.timedelta(days=1)

# محاسبه RFM
rfm = df.groupby('CustomerID').agg({
    'Date': lambda x: (NOW - x.max()).days,
    'Transaction_ID': 'nunique',
    'TotalPrice': 'sum'
})

rfm.rename(columns={
    'Date': 'Recency',
    'Transaction_ID': 'Frequency',
    'TotalPrice': 'Monetary'
}, inplace=True)

# امتیازدهی RFM با حذف خطای لبه‌های تکراری
rfm['R_Score'] = pd.qcut(rfm['Recency'], 5, labels=range(5, 0, -1), duplicates='drop')
rfm['F_Score'] = pd.qcut(rfm['Frequency'], 5, labels=range(1, 6), duplicates='drop')
rfm['M_Score'] = pd.qcut(rfm['Monetary'], 5, labels=range(1, 6), duplicates='drop')

rfm['RFM_Segment'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)
rfm['RFM_Score'] = rfm[['R_Score','F_Score','M_Score']].astype(int).sum(axis=1)

print(rfm)


             Recency  Frequency  Monetary R_Score F_Score M_Score RFM_Segment  \
CustomerID                                                                      
Customer_1        10          4      1144       5       4       5         545   
Customer_10      144          2       384       2       2       2         222   
Customer_11       17          1       172       5       1       1         511   
Customer_12      199          2       655       1       2       3         123   
Customer_13       34          3      1139       4       3       5         435   
Customer_14      109          2       515       3       2       3         323   
Customer_15       81          4       943       3       4       4         344   
Customer_16      170          1       313       2       1       1         211   
Customer_17      204          1       358       1       1       2         112   
Customer_19      173          2       601       1       2       3         123   
Customer_2       101        